In [1]:
from pyquery import PyQuery as pq
import requests
import pymysql
import json

## 爬取微博 电影台词的发博

### 首先找到基准网址

In [2]:
#设置请求头
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'Host':'m.weibo.cn',
    "Referer":'https://m.weibo.cn/p/1005051752054291',
    'X-Requested-With': 'XMLHttpRequest'
}

In [7]:
base_url = 'https://m.weibo.cn/api/container/getIndex?containerid=1076031752054291&page='

### 测试基准地址

In [9]:
link = base_url + str(0)
response = requests.get(link,headers=headers)
print(response.json())

{'ok': 1, 'data': {'cardlistInfo': {'containerid': '1076031752054291', 'v_p': 42, 'show_style': 1, 'total': 5522, 'page': 2}, 'cards': [{'card_type': 9, 'itemid': '1076031752054291_-_4407447793507871', 'scheme': 'https://m.weibo.cn/status/I37KzeIyz?mblogid=I37KzeIyz&luicode=10000011&lfid=1076031752054291', 'mblog': {'created_at': '昨天 19:35', 'id': '4407447793507871', 'idstr': '4407447793507871', 'mid': '4407447793507871', 'can_edit': False, 'show_additional_indication': 0, 'text': '心口烦闷的时候<br />大口喝酒最好<br /><br />——「全裸导演 第一季」 ', 'textLength': 52, 'source': '微博 weibo.com', 'favorited': False, 'pic_types': '', 'thumbnail_pic': 'http://wx2.sinaimg.cn/thumbnail/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg', 'bmiddle_pic': 'http://wx2.sinaimg.cn/bmiddle/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg', 'original_pic': 'http://wx2.sinaimg.cn/large/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg', 'is_paid': False, 'mblog_vip_type': 0, 'user': {'id': 1752054291, 'screen_name': '电影台词', 'profile_image_url': 'https://tvax2.si

得到响应数据的json格式的文件

In [39]:
x = response.json()
x

{'ok': 1,
 'data': {'cardlistInfo': {'containerid': '1076031752054291',
   'v_p': 42,
   'show_style': 1,
   'total': 5522,
   'page': 2},
  'cards': [{'card_type': 9,
    'itemid': '1076031752054291_-_4407447793507871',
    'scheme': 'https://m.weibo.cn/status/I37KzeIyz?mblogid=I37KzeIyz&luicode=10000011&lfid=1076031752054291',
    'mblog': {'created_at': '昨天 19:35',
     'id': '4407447793507871',
     'idstr': '4407447793507871',
     'mid': '4407447793507871',
     'can_edit': False,
     'show_additional_indication': 0,
     'text': '心口烦闷的时候<br />大口喝酒最好<br /><br />——「全裸导演 第一季」 ',
     'textLength': 52,
     'source': '微博 weibo.com',
     'favorited': False,
     'pic_types': '',
     'thumbnail_pic': 'http://wx2.sinaimg.cn/thumbnail/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg',
     'bmiddle_pic': 'http://wx2.sinaimg.cn/bmiddle/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg',
     'original_pic': 'http://wx2.sinaimg.cn/large/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg',
     'is_paid': False,
     'mblog_

获取核心信息

In [45]:
x.get('data').get('cards')[0].get('mblog')

{'created_at': '昨天 19:35',
 'id': '4407447793507871',
 'idstr': '4407447793507871',
 'mid': '4407447793507871',
 'can_edit': False,
 'show_additional_indication': 0,
 'text': '心口烦闷的时候<br />大口喝酒最好<br /><br />——「全裸导演 第一季」 ',
 'textLength': 52,
 'source': '微博 weibo.com',
 'favorited': False,
 'pic_types': '',
 'thumbnail_pic': 'http://wx2.sinaimg.cn/thumbnail/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg',
 'bmiddle_pic': 'http://wx2.sinaimg.cn/bmiddle/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg',
 'original_pic': 'http://wx2.sinaimg.cn/large/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg',
 'is_paid': False,
 'mblog_vip_type': 0,
 'user': {'id': 1752054291,
  'screen_name': '电影台词',
  'profile_image_url': 'https://tvax2.sinaimg.cn/crop.0.0.512.512.180/686e3a13ly8g4vd1naa6zj20e80e8mx3.jpg?KID=imgbed,tva&Expires=1566402168&ssig=GyuwPOpzD1',
  'profile_url': 'https://m.weibo.cn/u/1752054291?uid=1752054291&luicode=10000011&lfid=1076031752054291',
  'statuses_count': 5517,
  'verified': True,
  'verified_type': 0,
  'v

### 定义函数进行包装

In [46]:
def getPage(index):
    link = base_url + str(index)
    response = requests.get(link,headers=headers)
    if response.status_code == 200:
        print('Page',index,"get sucessful!")
        return response.json()
    else:
        print('Falied!')

In [57]:
def ParsePage(page):
    if page:
        items = page.get('data').get('cards')
        for item in items:
            weibo = {}
            weibo['time'] = item.get('mblog').get('created_at')
            weibo['text'] = pq(item.get('mblog').get('text')).text().replace('\n',' ')
            weibo['picture'] = item.get('mblog').get('original_pic')
            yield weibo

### 查看函数运行的结果

In [58]:
a = ParsePage(getPage(0))
for each in a:
    print(each)

Page 0 get sucessful!
{'time': '昨天 19:35', 'text': '心口烦闷的时候 大口喝酒最好  ——「全裸导演 第一季」', 'picture': 'http://wx2.sinaimg.cn/large/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg'}
{'time': '11分钟前', 'text': '#请喝汤#  趁世界末日没来之前 做现在想做的事', 'picture': 'http://wx1.sinaimg.cn/large/686e3a13ly1g679a7fe9hj21c00u0e85.jpg'}
{'time': '3小时前', 'text': '#丧句#  感觉我现在的生活 跟我想要的生活之间有点出入  为了电影超话', 'picture': 'http://wx2.sinaimg.cn/large/686e3a13ly1g678yzv3w6j216y0o60yx.jpg'}
{'time': '6小时前', 'text': '请给我来一打“大蒜”', 'picture': None}
{'time': '8小时前', 'text': '#午安#  就算是家人 也未必都有紧密的关系', 'picture': 'http://wx2.sinaimg.cn/large/686e3a13ly1g674up5l2yj21c00u0kjn.jpg'}
{'time': '9小时前', 'text': '#黑客帝国拍第四部#  经典科幻片《黑客帝国》系列将迎来第四部。  华纳正式宣布正在开发#黑客帝国4#，原班人马回归。#基努·里维斯#将继续饰演Neo。', 'picture': 'http://wx1.sinaimg.cn/large/686e3a13gy1g674gahevlj20go0nvdim.jpg'}
{'time': '10小时前', 'text': '“你不讨厌我了？” “我只是想和你待在一起。”  ——「请以你的名字呼唤我」', 'picture': 'http://wx1.sinaimg.cn/large/686e3a13gy1g674lrbozvj21c00rmjy4.jpg'}
{'time': '12小时前', 'text': '#早啊#  世界本就不公 你又能怎

### 开始爬取数据

In [60]:
weibosList = []
for i in range(20):#爬取最近20页的数据
    weibos = ParsePage(getPage(i))
    weibosList.extend(weibos)

Page 0 get sucessful!
Page 1 get sucessful!
Page 2 get sucessful!
Page 3 get sucessful!
Page 4 get sucessful!
Page 5 get sucessful!
Page 6 get sucessful!
Page 7 get sucessful!
Page 8 get sucessful!
Page 9 get sucessful!
Page 10 get sucessful!
Page 11 get sucessful!
Page 12 get sucessful!
Page 13 get sucessful!
Page 14 get sucessful!
Page 15 get sucessful!
Page 16 get sucessful!
Page 17 get sucessful!
Page 18 get sucessful!
Page 19 get sucessful!


### 查看数据是否正常

In [62]:
weibosList[0]

{'time': '昨天 19:35',
 'text': '心口烦闷的时候 大口喝酒最好  ——「全裸导演 第一季」',
 'picture': 'http://wx2.sinaimg.cn/large/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg'}

### 存储数据

In [85]:
pics = []
for each in weibosList:
    pics.append(each['picture'])
print('pics store sucess!')

pics store sucess!


In [89]:
l = pics[0]
l

'http://wx2.sinaimg.cn/large/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg'

### 重新定义请求头

In [95]:
h = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'host':'wx2.sinaimg.cn'
}

In [96]:
try:
    response = requests.get(l,headers=h)
except:
    print('error')

In [97]:
with open('movie_pics/text.jpg','wb') as f:
    f.write(response.content)
    print('ok')

ok


### 下载全部图片

In [99]:
for i in range(len(pics)):
    try:
        getpic = requests.get(pics[i],headers = h)
        with open('movie_pics/'+str(i)+'.jpg','wb') as f:
            f.write(getpic.content)
        print('picture' + str(i) + 'download ok!')
    except:
        print('picture' + str(i) + 'Failed!')   

picture0download ok!
picture1download ok!
picture2download ok!
picture3Failed!
picture4download ok!
picture5download ok!
picture6download ok!
picture7download ok!
picture8download ok!
picture9download ok!
picture10download ok!
picture11download ok!
picture12Failed!
picture13download ok!
picture14download ok!
picture15download ok!
picture16download ok!
picture17download ok!
picture18download ok!
picture19download ok!
picture20download ok!
picture21download ok!
picture22download ok!
picture23download ok!
picture24download ok!
picture25download ok!
picture26download ok!
picture27download ok!
picture28download ok!
picture29download ok!
picture30download ok!
picture31download ok!
picture32Failed!
picture33download ok!
picture34download ok!
picture35download ok!
picture36download ok!
picture37download ok!
picture38download ok!
picture39download ok!
picture40download ok!
picture41download ok!
picture42download ok!
picture43download ok!
picture44download ok!
picture45download ok!
picture46down

### 查看结果

![](../images/weiboPics.png)

### 存储全部博文

In [100]:
weibosList[0]

{'time': '昨天 19:35',
 'text': '心口烦闷的时候 大口喝酒最好  ——「全裸导演 第一季」',
 'picture': 'http://wx2.sinaimg.cn/large/686e3a13ly1g66d8vd23rj21c00u0b2c.jpg'}

In [103]:
with open('movie_say.txt','w',encoding='utf-8') as f:
    for each in weibosList:
        f.write('time: '+str(each['time']) +' 内容: '+ str(each['text']) + ' picture: ' + str(each['picture']) + '\n')
    print('write down sucessful!')

write down sucessful!


In [104]:
with open('movie_say.txt','w',encoding='utf-8') as f:
    for each in weibosList:
        f.write('time: '+str(each['time']) +' 内容: '+ str(each['text']) + '\n')
    print('write down sucessful!')

write down sucessful!


### 查看结果

![](../images/weiboContent.png)